In [ ]:
# 선형 회귀식은 nn.Linear()가 하나 있는 모델이다.
# nn.Linear()을 줄지어 여러층으로 구성된 신경망을 만들면 이를 다층 신경망 이라고 한다.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error # 회귀 문제의 평가를 위해 MSE 철도를 불러온 후 제곱근을 씌워 RMSE를 사용한다
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../../Pytorch_sample/data/reg.csv", index_col=[0])

In [ ]:
X = df.drop("Price", axis=1).to_numpy() # Price를 제외한 나머지를 변수로 사용한다.
Y = df["Price"].to_numpy().reshape((-1, 1))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

In [ ]:
# 텐서 데이터 만들기
class TensorData(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

trainsets = TensorData(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(trainsets, batch_size=32, shuffle=True)
testsets = TensorData(X_test, Y_test)
testloader = torch.utils.data.DataLoader(testsets, batch_size=32, shuffle=False)

In [ ]:
# 모델 구축하기
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13, 50, bias = True)
        self.fc2 = nn.Linear(50, 30, bias = True)
        self.fc3 = nn.Linear(30, 1, bias = True)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        x = F.relu(self.fc3(x))
        return x

In [ ]:
# 모델 학습
model = Regressor()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 1e-7)
# Adam은 최적화 방법을 정의한다. weight_decay는 L2정규화에서의 penalty 값을 의미하며, 값이 클수록 제약조건이 강함을 의미한다.

In [ ]:
loss_ = [] # 그래프를 그리기 위한 loss 저장용 리스트
n = len(trainloader) # 매 에포크 손실 함수값의 평균을 구하기 위해 배치 반복 수를 n이라고 한다.
for epoch in range(400):
    running_loss = 0.0 # 매 에포크 손실 함수값의 평균을 구하기 위해 초기값을 0으로 설정한다.
    for data in trainloader:
        inputs, values = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, values)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() # 매 에포크 손실 함수값의 평균을 구하기 위해 running_loss에 배치마다 로스를 더한다.

    loss_.append(running_loss / n)

In [ ]:
# 손실 함수 그리기
plt.plot(loss_)
plt.title("Training Loss")
plt.xlabel("epoch")
plt.show()

In [ ]:
# 모델 평가
def evaluation(dataloader):
    predictions = torch.tensor([], dtype=torch.float)
    actual = torch.tensor([], dtype=torch.float)
    with torch.no_grad():
        model.eval()
        for data in dataloader:
            inputs, values = data
            outputs = model(inputs)
            predictions = torch.cat((predictions, outputs), 0)
            actual = torch.cat((actual, values), 0)
            # torch.cat에서 0이라는 것은 0번째 차원을 기준으로 누적한다는 의미이다.
            # 즉, 10x2와 10x2인 두 텐서가 있을 때, cat의 기준이 0이면, 20x2가 되고 1이면 10x4가된다.
    predictions = predictions.numpy()
    actual = actual.numpy()
    rmse = np.sqrt(mean_squared_error(predictions, actual))

    return rmse

In [ ]:
train_rmse = evaluation(trainloader) # 학습 데이터의 RMSE
test_rmse = evaluation(testloader) # 시험 데이터의 RMSE
print(f"Train RMSE: {train_rmse}")
print(f"Test RMSE : {test_rmse}")

In [ ]:
# 학습 결과와 테스트 결과의 차이가 너무 크다.
# 즉, 학습 데이터에 과적합이 되어 있다고 판단할 수 있다.